In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import math

In [2]:
#dataset
dataset_train = pd.read_csv('../input/train.csv')
dataset_test = pd.read_csv('../input/test.csv')
print('Training Dataset samples:'+str(len(dataset_train)))
print('Samples to test:' + str(len(dataset_test)))

Training Dataset samples:42000
Samples to test:28000


Lets run a basic model on the dataset and see whats the performance on the test data


In [3]:
dataset_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Each Image is a 28 by 28 array of pixel values ranging from 0 to 255. First thing we will do is to normalize the training and test sets

In [4]:
#Training and test sets normalized
num_classes = 10
y_train = dataset_train['label'].values
x_train = dataset_train.drop(columns = ['label']).values/255.0
x_test = dataset_test.values/255.0
y_train = np.eye(num_classes)[y_train] #one hot encodings
print('Features shape:' + str(x_train.shape))
print('Number of samples: ' + str(y_train.shape[0]))
print('Number of samples to test:' + str(x_test.shape[0]))

Features shape:(42000, 784)
Number of samples: 42000
Number of samples to test:28000


## Architecture 
### 1. Input layer will consist of 784 units
### 2. There will be 2 hiddean layers of 512 and 128 units respectively
### 3. There will be 10 units in the output layer.
### 4. In hidden layers, there will be ReLu activation units. The output layer will have Softmax  activation, because we want the probabilities which will tell us that the input image represents which number.

#### The weights and biases matrices will have shapes as follows
W1----->(512,784), b1---->(1,512)
W2----->(128,512), b1---->(1,128)
W3----->(10,128),  b3---->(1,10)

In [5]:
#cost function
def cost_function(logits,labels):
    y = tf.placeholder(tf.float64,name='y')
    y_hat = tf.placeholder(tf.float64,name='y_hat')
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = labels,logits = logits))
    return cost

In [6]:
#create placeholders
def placeholders(num_features,num_classes):
    x = tf.placeholder(tf.float64,shape = [None, num_features],name = 'x_train')
    y = tf.placeholder(tf.float64,shape = [None,num_classes],name = 'y_train')
    return x,y

In [7]:
#initialize weights and biases as variables
def initialize_parameters():
    W1 = tf.get_variable(name = 'W1',shape = [512,784],dtype = tf.float64,initializer = tf.contrib.layers.xavier_initializer(seed = 1)) 
    b1 = tf.get_variable(name = 'b1',shape = [1,512],dtype = tf.float64,initializer = tf.initializers.zeros())
    W2 = tf.get_variable(name = 'W2',shape = [128,512],dtype = tf.float64,initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b2 = tf.get_variable(name = 'b2',shape = [1,128],dtype = tf.float64,initializer = tf.initializers.zeros())
    W3 = tf.get_variable(name = 'W3',shape = [10,128],dtype = tf.float64,initializer = tf.contrib.layers.xavier_initializer(seed = 1))
    b3 = tf.get_variable(name = 'b3',shape = [1,10],dtype = tf.float64,initializer = tf.initializers.zeros())
    
    params = { 'W1':W1,'b1':b1,
             'W2':W2,'b2':b2,
             'W3':W3,'b3':b3
            }
    return params 

We are not adding a softmax layer because our cost function computes the softmax then computes the output in a numerically stable way, for getting the output, we will softamx the logits of the last layer

In [8]:
#Forward propogation 
def forward_prop(x,vars):
    '''X is a placeholder containing the features batch or mini-batch'''
    W1,b1,W2,b2,W3,b3 = vars['W1'],vars['b1'],vars['W2'],vars['b2'],vars['W3'],vars['b3']
    z1 = tf.add(tf.matmul(x,tf.transpose(W1)),b1)
    x1 = tf.nn.relu(z1)
    z2 = tf.add(tf.matmul(x1,tf.transpose(W2)),b2)
    x2 = tf.nn.relu(z2)
    z3 = tf.add(tf.matmul(x2,tf.transpose(W3)),b3)
            
    return z3

In [9]:
#Creating mini batches
def data_loader(x_train,y_train,mini_batch_size):
        batches = []
        if(mini_batch_size=='default'):
            batches.append((x_train,y_train))
            return batches
        else:
            indices = np.arange(x_train.shape[0])
            np.random.shuffle(indices)
            x_tr,y_tr = np.copy(x_train),np.copy(y_train)
            x_tr = x_tr[indices]
            y_tr = y_tr[indices]
            num_batches = math.floor(x_train.shape[0]/mini_batch_size)
            for i in range(num_batches):
                batches.append((x_tr[i*mini_batch_size:(i+1)*mini_batch_size,:],y_tr[i*mini_batch_size:(i+1)*mini_batch_size,:]))
            if(x_train.shape[0]%mini_batch_size!=0):
                batches.append((x_tr[num_batches*mini_batch_size:,:],y_tr[num_batches*mini_batch_size:,:]))

            return batches

In [10]:
#Model building
def model(x_train,y_train,learning_rate,epochs,mini_batch_size='default'):
    costs = []
    tf.reset_default_graph()
    X,Y = placeholders(x_train.shape[1],y_train.shape[1])
    params = initialize_parameters()
    logits = forward_prop(X,params)
    cost = cost_function(logits,Y)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as session:
        session.run(init)
        
        for i in range(epochs):
            epoch_cost = 0
            mini_batches = data_loader(x_train,y_train,mini_batch_size)
            num_mini_batches = int(x_train.shape[0]/mini_batch_size)
            for b in mini_batches:
                x_batch,y_batch = b[0],b[1]
                _,c = session.run([optimizer,cost],feed_dict={X:x_batch,Y:y_batch})
                epoch_cost = epoch_cost + c
                
            if(i%10==0):
                print("Epoch: " + str(i))
                print("Training Cost: " + str(float(epoch_cost/num_mini_batches)))
                print("-------------------------------")
            costs.append(float(epoch_cost/num_mini_batches))
                
        trained_params = session.run(params)
        test_results = session.run(forward_prop(x_test,trained_params))
        
    return costs,trained_params,test_results

In [11]:
#Running on the discussed architecture
final_costs,trained_parameters,test_results = model(x_train,y_train,0.01,40,32)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Epoch: 0
Training Cost: 0.04607367067013939
-------------------------------
Epoch: 10
Training Cost: 0.011999790180242149
-------------------------------
Epoch: 20
Training Cost: 0.010509734447060382
-------------------------------
Epoch: 30
Training Cost: 0.009889466758035408
-------------------------------


In [12]:
#test set results to a dataframe
test_set_predictions_df = pd.DataFrame(columns=['ImageId','Label'],data={'ImageId':np.array(range(1,28001)),'Label':np.argmax(test_results,axis = 1)})
test_set_predictions_df.to_csv(path_or_buf='submission.csv',index = False)